# AR1 Observation report

## File to be processed

In [ ]:
filename = 

## Set up environment

In [ ]:
%pylab inline

In [ ]:
import katdal
import scape
import pickle
import katpoint

## Load data

In [ ]:
data = katdal.open(filename)

In [ ]:
N = data.shape[0]
ext = 930
step = max(int(floor(N/ext)),1)
print step

In [ ]:
print data

In [ ]:
data.obs_params

## Load static flags

In [ ]:
M = data.shape[1] / 4096
pickle_file = open('/var/kat/katsdpscripts/RTS/rfi_mask.pickle')
rfi_static_flags = pickle.load(pickle_file)
pickle_file.close()
edge = np.tile(True,4096)
edge[slice(211,3896)] = False
static_flags = np.logical_or(edge,rfi_static_flags)
static_flags = repeat(static_flags,M)
start_chan = 2200*M
end_chan = 2800*M

## Plot timeseries, spectrograph and spectrum for each antenna

In [ ]:
data.select()
ants = data.ants
for ant in ants:
    for active_pol in ['H','V']:
        ts_start = data.timestamps[0]
        ts_end = data.timestamps[-1]
        data.select(ants=ant.name,corrprods='auto',channels=~static_flags,pol=active_pol,dumps=slice(0,N,step),scans=['scan','track'])
        d = scape.DataSet(data)
        ts = data.timestamps - data.timestamps[0]
        figure(figsize=[15,10])
        subplot(221)
        scape.plot_xyz(d,'time','amp')
        title('Time series - Ant: %s Pol: %s'%(ant.name,active_pol))
        #yscale('log')
        ylim(1,2000)
        subplot(222)
        scape.plot_xyz(d,'time','freq','amp')
        title('Spectrogram - Ant: %s Pol: %s'%(ant.name,active_pol))
        subplot(223)
        scape.plot_xyz(d,'freq','amp',label='Average')
        title('Spectrum - Ant: %s Pol: %s'%(ant.name,active_pol))
        ylim(0,2000)
        subplot(224)
        scape.plot_xyz(d,'az','el')
        title('Pointing - Ant: %s Pol: %s'%(ant.name,active_pol))
    
    


In [ ]:
for active_pol in ['H','V']:
    data.select()
    data.select(corrprods='cross', channels=slice(start_chan,end_chan),pol=active_pol,scans=['scan','track'])
    baseline_names = [('%s - %s' % (inpA[:-1], inpB[:-1])) for inpA, inpB in data.corr_products]
    num_bls = data.shape[2]
    num_chans, sample_period = len(data.channel_freqs), data.dump_period
    scan_targets, scan_mid_az, scan_mid_el, scan_timestamps, scan_phase = [], [], [], [], []
    for scan_ind, state, target in data.scans():
        num_ts = data.shape[0]
        ts = data.timestamps[0::step]
        vis = data.vis[0::step,:,:]
        scan_phase.append(np.angle(vis).T.reshape(-1, vis.shape[0]))
        scan_timestamps.append(ts - data.start_time.secs)
    scan_freqinds = [np.arange(num_bls * num_chans)] * len(scan_timestamps)

    figure(figsize=(15,30))
    segms, labels, lines = scape.plots_basic.plot_segments(scan_timestamps, scan_freqinds, scan_phase, labels=scan_targets)
    for label in labels:
        label.set_rotation('vertical')
    plt.xlabel('Time (s), since %s' % (katpoint.Timestamp(data.start_time).local(),))
    plt.yticks(np.arange(num_chans // 2, num_bls * num_chans, num_chans), baseline_names)
    for yval in range(0, num_bls * num_chans, num_chans):
        plt.axhline(yval, color='k', lw=2)
    plt.title('Raw visibility phase per baseline, Pol %s'%active_pol)

In [ ]:
print 'Report generated:'
!date

In [ ]:
data.timestamps[1] - data.timestamps[0]

In [ ]:
for ant in ants:
    print ant.description

In [ ]:
print "Scape version: %s"%scape.__version__
print "katdal version: %s"%katdal.__version__
print "katpoint version: %s"%katpoint.__version__